# Atomic simulation environment:
[ase](https://wiki.fysik.dtu.dk/ase/)

In [ ]:
!pip install ase

In [1]:
# Imports
from ase import Atoms
from ase.optimize import BFGS
from ase.calculators.nwchem import NWChem
from ase.io import write

In [5]:
def atoms_to_html(atoms):
    'Return the html representation the atoms object as string'

    from tempfile import NamedTemporaryFile

    with NamedTemporaryFile('r+', suffix='.html') as ntf:
        atoms.write(ntf.name, format='html')
        ntf.seek(0)
        html = ntf.read()
    return html

### Displaying molecules

In [10]:
from IPython.display import HTML
from  ase.build import  molecule
tbut = molecule('trans-butane') 
tbut_html = atoms_to_html(tbut)
HTML(tbut_html)

### Example: structure optimization of hydrogen molecule

In [2]:
h2 = Atoms('H2',
...  positions=[[0, 0, 0],
...         [0, 0, 0.7]])

In [3]:
h2

Atoms(symbols='H2', pbc=False)

In [8]:
H2_html = atoms_to_html(h2)
HTML(H2_html)

### Water

In [11]:
from ase.constraints import FixBondLengths
from ase.calculators.tip3p import TIP3P, rOH, angleHOH
from ase.md import Langevin
import ase.units as units
from ase.io.trajectory import Trajectory
import numpy as np

See: [https://wiki.fysik.dtu.dk/ase/tutorials/tipnp_equil/tipnp_equil.html](https://wiki.fysik.dtu.dk/ase/tutorials/tipnp_equil/tipnp_equil.html)

In [ ]:
# Set up water box at 20 deg C density
x = angleHOH * np.pi / 180 / 2
pos = [[0, 0, 0],
       [0, rOH * np.cos(x), rOH * np.sin(x)],
       [0, rOH * np.cos(x), -rOH * np.sin(x)]]
atoms = Atoms('OH2', positions=pos)

vol = ((18.01528 / 6.022140857e23) / (0.9982 / 1e24))**(1 / 3.)
atoms.set_cell((vol, vol, vol))
atoms.center()

atoms = atoms.repeat((3, 3, 3))
atoms.set_pbc(True)

# RATTLE-type constraints on O-H1, O-H2, H1-H2.
atoms.constraints = FixBondLengths([(3 * i + j, 3 * i + (j + 1) % 3)
                                    for i in range(3**3)
                                    for j in [0, 1, 2]])

tag = 'tip3p_27mol_equil'
atoms.calc = TIP3P(rc=4.5)
md = Langevin(atoms, 1 * units.fs, temperature=300 * units.kB,
              friction=0.01, logfile=tag + '.log')

traj = Trajectory(tag + '.traj', 'w', atoms)
md.attach(traj.write, interval=1)
md.run(100)

# Repeat box and equilibrate further.
tag = 'tip3p_216mol_equil'
atoms.set_constraint()  # repeat not compatible with FixBondLengths currently.
atoms = atoms.repeat((2, 2, 2))
atoms.constraints = FixBondLengths([(3 * i + j, 3 * i + (j + 1) % 3)
                                    for i in range(len(atoms) / 3)
                                    for j in [0, 1, 2]])
atoms.calc = TIP3P(rc=7.)
md = Langevin(atoms, 2 * units.fs, temperature=300 * units.kB,
              friction=0.01, logfile=tag + '.log')

traj = Trajectory(tag + '.traj', 'w', atoms)
md.attach(traj.write, interval=1)
for i in range(20):
    md.run(10)
    printenergy(atoms)

In [13]:
H2O_html = atoms_to_html(atoms)
HTML(H2O_html)